In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

#tb.loc[tb['Instrument Classification (ISO 10962 CFI code)'].isin(['JFTXFP'])]
#tb.plot.hist(tb.groupby(['Trading platform']).Price.agg(['mean', max]))
#tb.describe()
#print(tb.shape)

df = pd.read_feather('C:/Users/asus/Documents/rts27.feather')

#Columns deemed unneccessary for this task
df2 = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)


In [4]:
#Columns deemed unneccessary for this task
#df = df.drop(['file_name','designated_market_makers','legal_entity_name', 'systematic_internaliser_flag'], axis = 1)
df = df2

#Filters the code to FX forwards only
#df = df.loc[(df['source_instrument_desc']).astype(str).str.contains("Forward")]
df = df.loc[(df['instrument_classification']).astype(str).str.contains("JF") | (df['source_instrument_desc']).astype(str).str.contains("Forward")]
#Drops the columns of which half of the fields have no data.
#df = df.dropna(axis=1, thresh=2000000)
for col in df.columns:
    #print(col, df[col].dtypes)
    if df[col].dtypes == 'int'or df[col].dtypes == 'float64' or df[col].dtypes == 'datetime64[ns]':
        df[col].fillna(0,  inplace=True)
    else: 
        df[col].fillna("NaN",  inplace=True)
        

C:\Users\asus\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Select target
y = df['date_created']
# To keep things simple, we'll use only numerical predictors
df = df.drop(['date_created'], axis=1)
X = df

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

y.head(10)

59    2019-05-13 14:24:39
61    2019-05-13 14:24:39
72    2019-05-13 14:24:39
73    2019-05-13 14:24:39
100   2019-01-15 15:01:55
101   2019-01-15 15:01:55
102   2019-01-15 15:01:55
103   2019-01-15 15:01:55
104   2019-01-15 15:01:55
105   2019-01-15 15:01:55
Name: date_created, dtype: datetime64[ns]

In [6]:
from sklearn.preprocessing import LabelEncoder
import bisect

# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

#print("Categorical variables:")
#print(object_cols)

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder to each column with categorical data
le = LabelEncoder()
for col in object_cols:
    le = LabelEncoder()
    le.fit(df[col])
    label_X_train[col] = le.transform(X_train[col])
    label_X_valid[col] = le.transform(X_valid[col])


In [8]:
train = label_X_train.values #returns a numpy array
valid = label_X_valid.values #returns a numpy array

min_max_scaler = preprocessing.MinMaxScaler()
train_scaled = min_max_scaler.fit_transform(train)
valid_scaled = min_max_scaler.transform(valid) #returns a numpy array
norm_train = pd.DataFrame(train_scaled)
norm_valid = pd.DataFrame(valid_scaled)

TypeError: float() argument must be a string or a number, not 'Timestamp'